In [50]:
import cv2, collections
import numpy as np
import matplotlib.pyplot as plt
import math
# import seaborn as sns
from queue import Queue
import sys

In [ ]:
vidObj = cv2.VideoCapture("../images/2016_04_07_07_09_21FrontColor_IN/2016_04_07_07_09_21FrontColor_IN.avi")
count = 0
success = 1
sec = 0
while success:
    vidObj.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    success, image = vidObj.read()
    if success == 1:
        cv2.imwrite("../images/2016_04_07_07_09_21FrontColor_IN/frames_0.25/frame%d.jpg" % count, image)
    count += 1
    sec += 0.25

In [12]:
feature_params = dict(maxCorners = 300, qualityLevel = 0.2, minDistance = 2, blockSize = 7)
lk_params = dict(winSize = (15,15), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
color = (0, 255, 0)
first_frame = cv2.imread("../images/2016_04_07_07_09_21FrontColor_IN/frames/frame0.jpg")
# first_frame = cv2.imread("images/trial1/frame60.jpg")

prev_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)
mask = np.zeros_like(first_frame)

T = 7.5          # threshold on the amount of movement recognized
K = 100            # the amount of seeds chosen for bfs

# takes position(i,j), visited matrix, motion vector matrix and threshold on vector magnitude
def bfs(i, j, vis, mag, t, region):
#     print(i,j)
    util = [-1, 0, 1]
    if mag[i][j] < t or vis[i][j] == True:
        return 0
    q = collections.deque()
    area = 0
    vis[i][j] = True
    q.append([i,j])
    while len(q) > 0:
        curX, curY = q.popleft()
        region[curX][curY] = 255
        area += 1
        for a in range(3):
            for b in range(3):
                x = curX + util[a]
                y = curY + util[b]
                if x >= 0 and y >= 0 and x < mag.shape[0] and y < mag.shape[1] and vis[x][y] != True:
                    vis[x][y] = True  
                    if mag[x][y] >= t:
                        q.append([x, y])
    return area
    
# plot = plt.figure(figsize=(100,100))

for index in range(1,144, 2):
    frame = cv2.imread("../images/2016_04_07_07_09_21FrontColor_IN/frames/frame"+ str(index)+".jpg")
#     frame = cv2.imread("images/trial1/frame"+ str(i)+".jpg")
    
    # plot.add_subplot(10, 2, 2*index-1)
    # plt.imshow(prev_gray, cmap='gray')
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    flow = gray.copy()
    prev = cv2.goodFeaturesToTrack(prev_gray, mask = None, **feature_params)

    # motion estimation
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 1, 5, 2, 5, 1.1, None)
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # considering only vertical motion
    vertical = mag.copy()
    for i in range(mag.shape[0]):
        for j in range(mag.shape[1]):
            vertical[i][j] = mag[i][j] * abs(np.sin(ang[i][j]))

    # calculating varying T
    ind = 0
    temp_array = [0]*(vertical.shape[0]*vertical.shape[1])
    for i in range(vertical.shape[0]):
        for j in range(vertical.shape[1]):
            temp_array[ind] = vertical[i][j]
            ind += 1
    temp_array = np.array(temp_array)
    temp_array = np.sort(temp_array)[::-1]
    # top 5%
    percent = (2*mag.shape[0]*mag.shape[1])/100
    percent = int(percent)
    print(temp_array[percent-1])
    # ax = sns.heatmap(mag)
    # plt.show()
    T = temp_array[percent-1]
    if T < 2:
        T = 100
    # Calculating feature value M
    dy, dx = np.gradient(gray)
    Ixx = dx**2
    Iyy = dy**2
    Ixy = dx*dy
    R_score = np.array(gray.copy(),dtype=np.float64)
    for i in range(gray.shape[0]):
        for j in range(gray.shape[1]):
            sum_Ix = 0
            sum_Iy = 0
            sum_Ixy = 0
            for k in range(-2,3):
                for l in range(-2,3):
                    if i+k >= 0 and j+l >= 0 and i+k < gray.shape[0] and j+l < gray.shape[1]:
                        sum_Ix += Ixx[i+k][j+l]
                        sum_Iy += Iyy[i+k][j+l]
                        sum_Ixy += Ixy[i+k][j+l]
                    else:
                        sum_Ix += 0
                        sum_Iy += 0
                        sum_Ixy += 0
            det = sum_Ix*sum_Iy - sum_Ixy*sum_Ixy
            trace = sum_Ixy + sum_Ixy
            r = det - 0.05*(trace**2)
            R_score[i][j] = r 
    min_val = np.min(R_score)
    R_score += (-1)*min_val
    max_val = np.max(R_score)
    R_score = (R_score/max_val)*255
    R_score = R_score.astype(int)
    
#     print(np.max(ang), np.min(ang), "angles")
    # ax = sns.heatmap(R_score)
    # plt.show()
    listOfFeatures = []
    listTemp = []
    for x in range(frame.shape[0]):
        for y in range(frame.shape[1]):
            listTemp.append([R_score[x][y], x, y])
    listTemp.sort()
    listTemp.reverse()
#     print(listTemp[0],listTemp[-1])
    for x in range(len(listTemp)):
        listOfFeatures.append(listTemp[x])
    
    # break
    region = np.zeros((frame.shape[0], frame.shape[1]), dtype=int)
    vis = [[False for x in range(frame.shape[1])] for y in range(frame.shape[0])]
    for point in listOfFeatures:
        area = bfs(point[1], point[2], vis, vertical, T, region)
#         print(np.max(region), np.min(region))
        # do something with the area if we want to
    # plot.add_subplot(10, 2, 2*index)
    # plt.imshow(region, cmap='gray')
#     plt.show()
    
#     break
    prev_gray = gray.copy()
#     prev = good_new.reshape(-1, 1, 2)
    mask = np.zeros_like(frame)
    plt.imsave('../images/2016_04_07_07_09_21FrontColor_IN/motion_vec_2_2/frame'+ str(index)+'.jpg',region)

2.6827502
4.4445148
7.863977
9.252568
8.694315
8.235002
8.527351
9.371463
11.776148
12.109111
7.54924
7.390139
6.4556203
6.9240317
3.9379864
1.2962892
1.1365741
1.0704855
1.1750033
1.1028562
1.0130306
1.2104867
1.2885653
1.3960872
1.3306733
1.7229364
2.4590142
2.816727
3.2530286
4.6766148
5.7143946
6.092952
6.691824
6.721398
6.949635
8.477892
7.561035
6.887634
4.630134
4.7302194
5.5243587
3.437428
4.3773556
6.3560977
6.677526
8.796921
10.405848
9.265533
5.938153
2.9536173
1.6205734
1.1062769
0.9942525
1.0465083
1.068734
1.0544109
1.0874391
0.98211515
1.0044208
0.9428092
0.8711306
0.8849765
0.9428419
1.0093352
0.9886778
0.9595967
0.9368776
0.98977214
1.0416278
1.1212714
1.2800866
1.4688933


In [14]:
x_value = [50, 130, 150, 200]
y_value = [100, 120, 130, 50]
in_below_100 = 0
in_above_100 = 0
out_above_170 = 0
out_below_170 = 0
in_person = 0
out_person = 0
for i in range(len(x_value)):
    x = x_value[3-i]
    y = y_value[i]
    if x < 100 and y < 210:
        if out_below_170 == 1:
            out_person += 1
            out_below_170 = 0
        else:
            in_below_100 = 1
    elif x > 100 and x < 170 and y < 210:
        if in_below_100 == 1:
            in_below_100 = 0
            in_above_100 = 1
        else:
            if out_above_170 == 1:
                out_above_170 = 0
                out_below_170 = 1
    elif x > 170 and y < 210:
        if in_above_100 == 1:
            in_above_100 = 0
            in_person += 1
        else:
            out_above_170 = 1
    print("in-person count-  " ,in_person)
    print("out-person count-  " ,out_person)

in-person count-   0
out-person count-   0
in-person count-   0
out-person count-   0
in-person count-   0
out-person count-   0
in-person count-   0
out-person count-   1


In [18]:
def centroid_bfs(img,r,c,col,vis):
    h,w = img.shape
    q = Queue()
    q.put((r,c))
    vis[r,c] = 1
    cent_x = 0
    cent_y = 0
    cnt = 0
    min_x = h+1
    min_y = w+1
    max_x = 0
    max_y = 0
    while not q.empty():
        r, c = q.get()
        min_x = min(min_x,r)
        min_y = min(min_y,c)
        max_x = max(max_x,r)
        max_y = max(max_y,c)
        cnt += 1
        cent_x += r
        cent_y += c
        if r-1>=0 and vis[r-1,c] == 0 and img[r-1,c]!=0:
            vis[r-1,c] = col
            q.put((r-1,c))
        if r+1<h and vis[r+1,c] == 0 and img[r+1,c]!=0:
            vis[r+1,c] = 1
            q.put((r+1,c))
        if c-1>=0 and vis[r,c-1] == 0 and img[r,c-1]!=0:
            vis[r,c-1] = 1
            q.put((r,c-1))
        if c+1<w and vis[r,c+1] == 0 and img[r,c+1]!=0:
            vis[r,c+1] = 1
            q.put((r,c+1))
            
    cent_x = cent_x//cnt
    cent_y = cent_y//cnt
    
    # bounding box co-ordinates are pushed in an array
    arr = [(min_x,min_y),(min_x,max_y),(max_x,min_y),(max_x,max_y)]
    
    return cent_x, cent_y, arr

In [38]:
def centroid_plotting(col_img_path):
    img = cv2.imread(col_img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # plt.imshow(img, cmap="gray")
    # plt.show()
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if img[i][j] > 50:
                img[i][j] = 1
            else:
                img[i][j] = 0
    
    # plt.imshow(img, cmap="gray")
    # return 
    # break
    cent = []
    vis = np.zeros(img.shape)
    h, w= img.shape
    for i in range(h):
        for j in range(w):
            if img[i,j]==1 and vis[i,j]==0:
                # the cent_x and cent_y has x,y coordi of centroid and the array arr has the bounding box coordi                 
                cent_x, cent_y, arr = centroid_bfs(img,i,j,1,vis)
                cent.append((cent_x,cent_y,arr))
    return cent

In [39]:
cent = centroid_plotting("../images/2016_04_07_07_09_21FrontColor_IN/joined_2_with_0.25_2%/frame11.jpg")

In [48]:
in_below_100 = 0
in_above_100 = 0
out_above_170 = 0
out_below_170 = 0
in_person = 0
out_person = 0
for i in range(1, 144):
    centroid =   centroid_plotting("../images/2016_04_07_07_09_21FrontColor_IN/joined_2_with_0.25_2%/frame" + str(i) +".jpg")
    if len(centroid) == 0:
        if out_below_170 == 1:
            out_person += 1
            out_below_170 = 0
        if in_above_100 == 1:
            in_above_100 = 0
            in_person += 1
    for j in range(len(centroid)):
        x = centroid[j][0]
        y = centroid[j][1]
        if x < 100 and y < 210:
            if out_below_170 == 0:
                in_below_100 = 1
        elif x > 100 and x < 170 and y < 210:
            if in_below_100 == 1:
                in_below_100 = 0
                in_above_100 = 1
            else:
                if out_above_170 == 1:
                    out_above_170 = 0
                    out_below_170 = 1
        elif x > 170 and y < 210:
            if in_above_100 == 0:
                out_above_170 = 1
    print(i,"  in-person count-  " ,in_person, "out-person count-  " ,out_person)
        # print("out-person count-  " ,out_person)

1   in-person count-   0 out-person count-   0
2   in-person count-   0 out-person count-   0
3   in-person count-   0 out-person count-   0
4   in-person count-   0 out-person count-   0
5   in-person count-   0 out-person count-   0
6   in-person count-   0 out-person count-   0
7   in-person count-   0 out-person count-   0
8   in-person count-   0 out-person count-   0
9   in-person count-   0 out-person count-   0
10   in-person count-   0 out-person count-   0
11   in-person count-   0 out-person count-   0
12   in-person count-   0 out-person count-   0
13   in-person count-   1 out-person count-   0
14   in-person count-   1 out-person count-   0
15   in-person count-   1 out-person count-   0
16   in-person count-   1 out-person count-   0
17   in-person count-   1 out-person count-   0
18   in-person count-   1 out-person count-   0
19   in-person count-   1 out-person count-   0
20   in-person count-   1 out-person count-   0
21   in-person count-   1 out-person count-   0
2